In [13]:
from monomial_neural_network import *
from experiment import *

In [ ]:
d = 2 # input data dimension
teacher_k = [4] # teacher model hidden layer sizes - 3 layers with increasing number of neurons
# teacher_k = [10] # teacher model hidden layer sizes - 1 layer with 10 neurons

teacher_model = generate_teacher_model(d, teacher_k)
print(teacher_model)

n = 1000 # number of data points
# same data dimension d as before

data = generate_data(n, d, teacher_model)

# verify that the data is generated correctly
print(data[0][0], data[1][0], teacher_model.evaluate(data[0][0]))

student_k = [4] # student model hidden layer sizes - 2 layers with increasing number of neurons
student_model = generate_student_model(d, student_k)
print(student_model)



MonomialNeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=False)
    (1): Monomial()
    (2): Linear(in_features=3, out_features=1, bias=False)
  )
)
tensor([-0.6756,  0.1704]) tensor([1.5375]) tensor([1.5375])
MonomialNeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=2, out_features=3, bias=False)
    (1): Monomial()
    (2): Linear(in_features=3, out_features=1, bias=False)
  )
)


In [74]:
# train the student
student_model, losses = train(
    model = student_model, 
    x_train = data[0], 
    y_train= data[1], 
    num_epochs = 20000, 
    lr = 1e-3
    )

starting training
Epoch [0/20000], Loss: 42.20013
Epoch [100/20000], Loss: 37.63603
Epoch [200/20000], Loss: 33.15986
Epoch [300/20000], Loss: 28.69736
Epoch [400/20000], Loss: 24.23464
Epoch [500/20000], Loss: 19.81615
Epoch [600/20000], Loss: 15.54110
Epoch [700/20000], Loss: 11.56037
Epoch [800/20000], Loss: 8.07281
Epoch [900/20000], Loss: 5.31356
Epoch [1000/20000], Loss: 3.50147
Epoch [1100/20000], Loss: 2.57107
Epoch [1200/20000], Loss: 2.01316
Epoch [1300/20000], Loss: 1.63393
Epoch [1400/20000], Loss: 1.38510
Epoch [1500/20000], Loss: 1.22775
Epoch [1600/20000], Loss: 1.14757
Epoch [1700/20000], Loss: 1.12256
Epoch [1800/20000], Loss: 1.11463
Epoch [1900/20000], Loss: 1.11207
Epoch [2000/20000], Loss: 1.11124
Epoch [2100/20000], Loss: 1.11097
Epoch [2200/20000], Loss: 1.11088
Epoch [2300/20000], Loss: 1.11085
Epoch [2400/20000], Loss: 1.11084
Epoch [2500/20000], Loss: 1.11083
Epoch [2600/20000], Loss: 1.11083
Epoch [2700/20000], Loss: 1.11083
Epoch [2800/20000], Loss: 1.11083


In [75]:
# lets look at the weight matrices
# print(student_model.layers[0].weight)
student_w = student_model.layers[0].weight.detach().numpy()
student_v = student_model.layers[2].weight.detach().numpy()
# print(student_w)
teacher_w = teacher_model.layers[0].weight.detach().numpy()
teacher_v = teacher_model.layers[2].weight.detach().numpy()
# print(student_w)
# print(teacher_w)
# print(np.squeeze(student_v))
A = student_w.T @ np.diag(np.squeeze(student_v)) @ student_w
Astar = teacher_w.T @ np.diag(np.squeeze(teacher_v)) @ teacher_w
print(A)
print(Astar)
eigA, eigvecA = np.linalg.eig(A)
print(eigA)
# print(Astar)
eigAstar, eigvecAstar = np.linalg.eig(Astar)
print(eigAstar)
e = pop_loss(student_model, teacher_model, d=d, N=1000000)
# print(y_teach)
# print(y_stud)
# print(torch.norm(y_teach))
print(e)

[[ 2.938348  -1.1528543]
 [-1.1528543  0.4523199]]
[[ 2.8567715  -0.89306384]
 [-0.89306384  0.96261173]]
[3.390668e+00 5.272745e-08]
[3.21143  0.607953]
0.0009902802124023438


In [60]:
test_vec = np.array([[1,2]]).T
print(test_vec.T@A@test_vec)
# print(test_vec.T@Astar@test_vec)
test_vec_torch = torch.from_numpy(test_vec.T).type(torch.FloatTensor)
print(test_vec_torch)
print(student_model(test_vec_torch))
print(student_model.layers[2].weight)
print(teacher_model.layers[2].weight)
# print(teacher_model(test_vec_torch))

[[22.01623166]]
tensor([[1., 2.]])
tensor([[11.4299]], grad_fn=<MmBackward0>)
Parameter containing:
tensor([[-0.4712,  1.2445,  1.0397, -1.0011]], requires_grad=True)
Parameter containing:
tensor([[1., 1., 1., 1.]], requires_grad=True)
